# 🤖 股票关联分析智能体 (Stock Correlation Analysis Agent)

**功能概述**：
- 🧠 基于大语言模型的智能股票选择
- 🌍 支持多股市（美股、A股、港股、日股）
- 🔗 自动分析行业上下游关联
- 📊 生成股票列表JSON和分析报告
- 📈 自动抓取历史数据（按小时/按天）
- ⏱️ 智能分批和延迟，避免API限流

**支持的LLM**：
- DeepSeek - 默认（成本低，性价比高）
- Google AI (Gemini)
- OpenAI (GPT-4/GPT-3.5)
- 其他兼容OpenAI API的模型

---

**作者**: Quant-Stock-Transformer Team  
**版本**: 1.0.1  
**状态**: 🚧 开发测试中

## 📦 Step 1: 环境设置和依赖安装

In [ ]:
# 安装必要的依赖
!pip install -q google-generativeai openai akshare yfinance pandas numpy
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

print("✓ 依赖安装完成")

In [ ]:
import json
import os
import time
from datetime import datetime, timedelta
from typing import List, Dict, Optional, Literal
import pandas as pd
import numpy as np

# LLM相关
import google.generativeai as genai
from openai import OpenAI

# 数据获取
import akshare as ak  # A股数据
import yfinance as yf  # 美股、港股、日股

print("✓ 导入完成")

## 🔑 Step 2: LLM配置

**默认使用DeepSeek**（成本低，性价比高）

### 设置API Key（使用Colab Secrets - 推荐）

1. 点击左侧 🔑 **Secrets** 图标
2. 添加新secret：
   - Name: `DEEPSEEK_API_KEY`
   - Value: 你的DeepSeek API key
3. 运行下面的cell即可

**获取DeepSeek API Key**：
- 访问 https://platform.deepseek.com/
- 注册并充值（最低10元人民币）
- 创建API Key

**成本参考**：
- ~0.14元/M tokens
- 100次股票分析 < 0.1元

In [ ]:
class LLMConfig:
    """LLM配置类"""
    
    def __init__(
        self,
        provider: Literal["google", "openai", "deepseek", "custom"] = "deepseek",
        api_key: Optional[str] = None,
        model_name: Optional[str] = None,
        base_url: Optional[str] = None
    ):
        self.provider = provider
        self.api_key = api_key
        
        # 设置默认模型
        if model_name is None:
            self.model_name = {
                "google": "gemini-2.0-flash-exp",
                "openai": "gpt-4-turbo-preview",
                "deepseek": "deepseek-chat",
                "custom": "gpt-3.5-turbo"
            }[provider]
        else:
            self.model_name = model_name
        
        self.base_url = base_url
        
        # 初始化客户端
        self._init_client()
    
    def _init_client(self):
        """初始化LLM客户端"""
        if self.provider == "google":
            if not self.api_key:
                from google.colab import userdata
                try:
                    self.api_key = userdata.get('GOOGLE_API_KEY')
                except:
                    raise ValueError("请在Colab Secrets中设置GOOGLE_API_KEY")
            
            genai.configure(api_key=self.api_key)
            self.client = genai.GenerativeModel(self.model_name)
            print(f"✓ 使用Google AI: {self.model_name}")
        
        elif self.provider == "openai":
            if not self.api_key:
                from google.colab import userdata
                try:
                    self.api_key = userdata.get('OPENAI_API_KEY')
                except:
                    raise ValueError("请在Colab Secrets中设置OPENAI_API_KEY")
            
            self.client = OpenAI(api_key=self.api_key)
            print(f"✓ 使用OpenAI: {self.model_name}")
        
        elif self.provider == "deepseek":
            if not self.api_key:
                from google.colab import userdata
                try:
                    self.api_key = userdata.get('DEEPSEEK_API_KEY')
                except:
                    raise ValueError(
                        "请在Colab Secrets中设置DEEPSEEK_API_KEY\n"
                        "步骤：\n"
                        "1. 点击左侧🔑图标\n"
                        "2. 添加 Name: DEEPSEEK_API_KEY, Value: 你的API key"
                    )
            
            self.client = OpenAI(
                api_key=self.api_key,
                base_url="https://api.deepseek.com"
            )
            print(f"✓ 使用DeepSeek: {self.model_name}")
            print(f"  成本：~0.14元/M tokens，性价比极高")
        
        elif self.provider == "custom":
            self.client = OpenAI(
                api_key=self.api_key,
                base_url=self.base_url
            )
            print(f"✓ 使用自定义API: {self.model_name}")
    
    def generate(self, prompt: str) -> str:
        """生成文本"""
        if self.provider == "google":
            response = self.client.generate_content(prompt)
            return response.text
        else:
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7
            )
            return response.choices[0].message.content


# ============================================================================
# 配置你的LLM
# ============================================================================

# 默认选项：DeepSeek（使用Colab Secrets）
# 确保在左侧🔑Secrets中添加了 DEEPSEEK_API_KEY
llm_config = LLMConfig(provider="deepseek")

# 其他选项：
# llm_config = LLMConfig(provider="google")      # Google AI (需要GOOGLE_API_KEY)
# llm_config = LLMConfig(provider="openai")      # OpenAI (需要OPENAI_API_KEY)
# llm_config = LLMConfig(
#     provider="deepseek",
#     api_key="sk-xxxx"  # 直接提供API key（不推荐，不安全）
# )

### 🧪 测试LLM连接（可选）

In [ ]:
# 测试LLM是否配置成功
try:
    test_response = llm_config.generate("你好，请用一句话介绍你自己")
    print("✓ LLM连接成功！\n")
    print(f"响应: {test_response}")
except Exception as e:
    print(f"✗ 连接失败: {e}")
    print("\n请检查：")
    print("1. Colab Secrets中是否正确设置了API key")
    print("2. API key是否有效且有余额")

## 🤖 Step 3: 股票关联分析智能体

这个智能体会根据指定行业，自动分析并选择相关股票。

In [ ]:
class StockAnalysisAgent:
    """股票关联分析智能体"""
    
    def __init__(self, llm_config: LLMConfig):
        self.llm = llm_config
        self.selected_stocks = {}
        self.analysis_report = ""
    
    def analyze_industry(
        self,
        industry: str,
        markets: List[Literal["US", "CN", "HK", "JP"]],
        min_stocks_per_market: Dict[str, int] = None
    ) -> Dict:
        """
        分析指定行业并选择相关股票
        
        Args:
            industry: 行业名称（如"半导体"、"新能源汽车"）
            markets: 要分析的股市列表
            min_stocks_per_market: 每个市场的最小股票数量
        
        Returns:
            包含股票列表和分析报告的字典
        """
        if min_stocks_per_market is None:
            min_stocks_per_market = {market: 5 for market in markets}
        
        print(f"🔍 开始分析行业: {industry}")
        print(f"📊 目标市场: {', '.join(markets)}")
        print(f"📈 最小股票数: {min_stocks_per_market}")
        print("\n" + "="*80)
        
        # 构建prompt
        prompt = self._build_analysis_prompt(
            industry, markets, min_stocks_per_market
        )
        
        # 调用LLM
        print("🤖 正在调用LLM进行分析...")
        response = self.llm.generate(prompt)
        
        # 解析结果
        print("📝 正在解析分析结果...")
        result = self._parse_llm_response(response, markets)
        
        self.selected_stocks = result['stocks']
        self.analysis_report = result['report']
        
        print("\n✓ 分析完成！")
        print(f"\n选中的股票数量:")
        for market, stocks in self.selected_stocks.items():
            print(f"  {market}: {len(stocks)}只")
        
        return result
    
    def _build_analysis_prompt(self, industry, markets, min_stocks):
        """构建分析prompt"""
        
        market_names = {
            "US": "美国股市",
            "CN": "中国A股",
            "HK": "香港股市",
            "JP": "日本股市"
        }
        
        prompt = f"""
你是一位专业的股票分析师。请分析【{industry}】行业的股票关联关系。

# 任务要求

1. **上下游分析**：识别该行业的上游供应商、下游客户
2. **竞争对手**：找出主要竞争对手
3. **关联行业**：找出可能受影响的相关行业
4. **市场领导者**：识别各市场的龙头企业

# 目标市场和最小股票数量

"""
        for market in markets:
            prompt += f"- **{market_names[market]}**: 至少选择 {min_stocks[market]} 只股票\n"
        
        prompt += f"""

# 输出格式要求

请严格按照以下JSON格式输出（必须是有效的JSON）：

```json
{{
  "stocks": {{
"""
        for i, market in enumerate(markets):
            prompt += f'    "{market}": [\n'
            prompt += '      {"symbol": "股票代码", "name": "公司名称", "reason": "选择理由", "category": "上游/下游/竞争者/关联"},\n'
            prompt += '      ...\n'
            prompt += '    ]' + (',\n' if i < len(markets)-1 else '\n')
        
        prompt += f"""
  }},
  "analysis": {{
    "summary": "行业整体分析总结",
    "upstream": "上游分析",
    "downstream": "下游分析",
    "competitors": "竞争格局分析",
    "correlations": "关联性分析"
  }}
}}
```

# 股票代码格式说明

- **美股(US)**: 直接使用ticker（如NVDA, TSMC）
- **A股(CN)**: 使用6位代码（如600519, 000858）
- **港股(HK)**: 使用数字代码（如00700, 01810）
- **日股(JP)**: 使用4位代码.T（如6758.T, 8035.T）

# 注意事项

1. 确保每个市场的股票数量 >= 最小要求
2. 股票代码必须真实存在且可交易
3. 选择理由要具体，说明与{industry}行业的关联
4. 分析要深入，考虑产业链完整性

请开始分析并输出JSON结果：
"""
        return prompt
    
    def _parse_llm_response(self, response: str, markets: List[str]) -> Dict:
        """解析LLM返回的JSON"""
        
        # 提取JSON部分
        try:
            # 尝试找到JSON代码块
            if "```json" in response:
                json_str = response.split("```json")[1].split("```")[0].strip()
            elif "```" in response:
                json_str = response.split("```")[1].strip()
            else:
                json_str = response.strip()
            
            # 解析JSON
            data = json.loads(json_str)
            
            # 验证结构
            if "stocks" not in data or "analysis" not in data:
                raise ValueError("JSON格式不完整")
            
            # 构建报告
            report = self._generate_report(data, markets)
            
            return {
                "stocks": data["stocks"],
                "report": report,
                "raw_analysis": data["analysis"]
            }
        
        except Exception as e:
            print(f"\n❌ 解析失败: {e}")
            print(f"\n原始响应:\n{response}")
            raise
    
    def _generate_report(self, data: Dict, markets: List[str]) -> str:
        """生成分析报告"""
        
        report = f"""
# 股票关联分析报告

**生成时间**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## 📊 选股统计

"""
        for market in markets:
            stocks = data['stocks'].get(market, [])
            report += f"- **{market}**: {len(stocks)}只股票\n"
        
        report += f"""

## 🔍 行业分析

### 总体概述
{data['analysis'].get('summary', 'N/A')}

### 上游分析
{data['analysis'].get('upstream', 'N/A')}

### 下游分析
{data['analysis'].get('downstream', 'N/A')}

### 竞争格局
{data['analysis'].get('competitors', 'N/A')}

### 关联性分析
{data['analysis'].get('correlations', 'N/A')}

## 📋 股票明细

"""
        for market in markets:
            stocks = data['stocks'].get(market, [])
            if stocks:
                report += f"\n### {market}市场\n\n"
                for stock in stocks:
                    report += f"- **{stock['symbol']}** ({stock['name']})\n"
                    report += f"  - 类别: {stock.get('category', 'N/A')}\n"
                    report += f"  - 理由: {stock.get('reason', 'N/A')}\n\n"
        
        return report
    
    def save_results(
        self,
        json_path: str = "selected_stocks.json",
        report_path: str = "analysis_report.md"
    ):
        """保存结果到文件"""
        
        # 保存JSON
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(self.selected_stocks, f, ensure_ascii=False, indent=2)
        print(f"✓ JSON保存到: {json_path}")
        
        # 保存报告
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write(self.analysis_report)
        print(f"✓ 报告保存到: {report_path}")


print("✓ StockAnalysisAgent类定义完成")

## 🎯 Step 4: 运行智能体分析

配置你的分析参数并运行。

In [ ]:
# 创建智能体
agent = StockAnalysisAgent(llm_config)

# ============================================================================
# 配置分析参数（根据你的需求修改）
# ============================================================================

INDUSTRY = "半导体"  # 行业名称

MARKETS = ["US", "CN", "HK", "JP"]  # 要分析的市场
# 可选: ["US"], ["CN"], ["US", "CN"], ["US", "CN", "HK", "JP"]

MIN_STOCKS_PER_MARKET = {
    "US": 8,   # 美股至少8只
    "CN": 10,  # A股至少10只
    "HK": 5,   # 港股至少5只
    "JP": 5    # 日股至少5只
}

# ============================================================================
# 运行分析
# ============================================================================

result = agent.analyze_industry(
    industry=INDUSTRY,
    markets=MARKETS,
    min_stocks_per_market=MIN_STOCKS_PER_MARKET
)

# 保存结果
agent.save_results(
    json_path="selected_stocks.json",
    report_path="analysis_report.md"
)

print("\n" + "="*80)
print("\n📄 分析报告预览:\n")
print(agent.analysis_report[:500] + "...")

## 📈 Step 5: 历史数据抓取

根据选中的股票，自动抓取历史数据。

### ⏱️ 智能分批和延迟

**为什么需要分批和延迟？**
- AkShare和yfinance都有频率限制
- 避免被API服务器限流或封禁
- 提高数据获取的稳定性

**默认参数（推荐）**：
- 每批5只股票
- 批次间延迟2秒
- 股票间延迟0.5秒

**适用场景**：
- 几十支股票（30-50支）
- 一年的每日或小时数据
- 总耗时：约30-50秒（可接受）

In [ ]:
class StockDataFetcher:
    """多市场股票数据抓取器（支持智能分批和延迟）"""
    
    def __init__(self):
        self.data_cache = {}
    
    def fetch_historical_data(
        self,
        stocks_json: Dict,
        start_date: str,
        end_date: str,
        interval: Literal["1d", "1h"] = "1d",
        include_market_index: bool = True,
        batch_size: int = 5,
        delay_between_batches: float = 2.0,
        delay_between_stocks: float = 0.5
    ) -> Dict:
        """
        抓取历史数据（智能分批，避免API限流）
        
        Args:
            stocks_json: 股票JSON（agent.selected_stocks）
            start_date: 开始日期 (YYYY-MM-DD)
            end_date: 结束日期 (YYYY-MM-DD)
            interval: 数据粒度（"1d"按天，"1h"按小时）
            include_market_index: 是否包含大盘指数
            batch_size: 每批抓取的股票数量（默认5，推荐3-10）
            delay_between_batches: 批次间延迟秒数（默认2.0秒）
            delay_between_stocks: 同批股票间延迟秒数（默认0.5秒）
        
        Returns:
            包含所有股票和指数数据的字典
        
        推荐配置：
            - 几十支股票，一年数据：默认参数即可
            - 大量股票（>100支）：batch_size=3, delay_between_batches=3.0
            - 少量股票（<20支）：batch_size=10, delay_between_batches=1.0
        """
        
        print(f"\n📥 开始抓取历史数据")
        print(f"日期范围: {start_date} 至 {end_date}")
        print(f"数据粒度: {interval}")
        print(f"⏱️  分批配置: 每批{batch_size}支，批次间延迟{delay_between_batches}秒，股票间延迟{delay_between_stocks}秒")
        print("\n" + "="*80)
        
        all_data = {}
        
        for market, stocks in stocks_json.items():
            print(f"\n🔄 正在处理{market}市场 ({len(stocks)}只股票)...")
            
            market_data = {}
            
            # 抓取大盘指数
            if include_market_index:
                index_data = self._fetch_market_index(
                    market, start_date, end_date, interval
                )
                if index_data is not None:
                    market_data['_INDEX_'] = index_data
                    print(f"  ✓ 大盘指数数据获取成功 ({len(index_data)}条记录)")
                time.sleep(delay_between_stocks)  # 延迟后再抓取个股
            
            # 分批抓取个股数据
            total_stocks = len(stocks)
            num_batches = (total_stocks + batch_size - 1) // batch_size
            
            for batch_idx in range(num_batches):
                start_idx = batch_idx * batch_size
                end_idx = min((batch_idx + 1) * batch_size, total_stocks)
                batch_stocks = stocks[start_idx:end_idx]
                
                print(f"\n  批次 [{batch_idx+1}/{num_batches}]: 抓取第{start_idx+1}-{end_idx}支股票")
                
                for i, stock in enumerate(batch_stocks, start=start_idx+1):
                    symbol = stock['symbol']
                    try:
                        data = self._fetch_stock_data(
                            market, symbol, start_date, end_date, interval
                        )
                        if data is not None and len(data) > 0:
                            market_data[symbol] = data
                            print(f"    ✓ [{i}/{total_stocks}] {symbol}: {len(data)}条数据")
                        else:
                            print(f"    ✗ [{i}/{total_stocks}] {symbol}: 无数据")
                    except Exception as e:
                        print(f"    ✗ [{i}/{total_stocks}] {symbol}: 失败 ({str(e)[:50]})")
                    
                    # 同批股票间延迟
                    if i < total_stocks:
                        time.sleep(delay_between_stocks)
                
                # 批次间延迟（最后一批不需要）
                if batch_idx < num_batches - 1:
                    print(f"  ⏸️  批次完成，等待{delay_between_batches}秒后继续...")
                    time.sleep(delay_between_batches)
            
            all_data[market] = market_data
            print(f"\n  ✓ {market}市场完成：成功{len(market_data)}支（含指数）")
        
        self.data_cache = all_data
        
        print("\n" + "="*80)
        print("✓ 所有数据抓取完成！")
        
        # 统计
        total_success = sum(len(v) for v in all_data.values())
        total_requested = sum(len(v) for v in stocks_json.values()) + len(stocks_json)  # +指数
        print(f"\n成功率: {total_success}/{total_requested} ({100*total_success/total_requested:.1f}%)")
        
        return all_data
    
    def _fetch_market_index(
        self, market: str, start_date: str, end_date: str, interval: str
    ) -> Optional[pd.DataFrame]:
        """抓取大盘指数"""
        
        index_symbols = {
            "US": "^GSPC",      # S&P 500
            "CN": "000001",     # 上证指数
            "HK": "^HSI",       # 恒生指数
            "JP": "^N225"       # 日经225
        }
        
        symbol = index_symbols.get(market)
        if not symbol:
            return None
        
        try:
            if market == "CN":
                # A股使用akshare
                df = ak.stock_zh_index_daily(symbol=f"sh{symbol}")
                df['date'] = pd.to_datetime(df['date'])
                df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
                df = df.rename(columns={'close': 'Close', 'open': 'Open', 
                                       'high': 'High', 'low': 'Low', 'volume': 'Volume'})
            else:
                # 其他市场使用yfinance
                df = yf.download(symbol, start=start_date, end=end_date, 
                                interval=interval, progress=False)
            
            return df
        except Exception as e:
            print(f"    警告: 大盘指数获取失败 ({e})")
            return None
    
    def _fetch_stock_data(
        self, market: str, symbol: str, start_date: str, end_date: str, interval: str
    ) -> Optional[pd.DataFrame]:
        """抓取个股数据"""
        
        try:
            if market == "CN":
                # A股使用akshare
                df = ak.stock_zh_a_hist(symbol=symbol, period="daily", 
                                       start_date=start_date.replace('-', ''),
                                       end_date=end_date.replace('-', ''))
                df['日期'] = pd.to_datetime(df['日期'])
                df = df.rename(columns={
                    '日期': 'Date', '收盘': 'Close', '开盘': 'Open',
                    '最高': 'High', '最低': 'Low', '成交量': 'Volume'
                })
                df = df.set_index('Date')
            else:
                # 其他市场使用yfinance
                # 确保symbol格式正确
                if market == "HK" and not symbol.endswith(".HK"):
                    symbol = symbol.zfill(4) + ".HK"
                elif market == "JP" and not symbol.endswith(".T"):
                    if '.' not in symbol:
                        symbol = symbol + ".T"
                
                df = yf.download(symbol, start=start_date, end=end_date,
                                interval=interval, progress=False)
            
            return df
        except Exception as e:
            raise Exception(f"数据获取失败: {e}")
    
    def save_data(
        self,
        output_path: str = "historical_data.pkl"
    ):
        """保存数据到pickle文件"""
        import pickle
        
        with open(output_path, 'wb') as f:
            pickle.dump(self.data_cache, f)
        
        print(f"✓ 数据已保存到: {output_path}")
    
    @staticmethod
    def load_data(input_path: str) -> Dict:
        """从pickle文件加载数据"""
        import pickle
        
        with open(input_path, 'rb') as f:
            data = pickle.load(f)
        
        print(f"✓ 数据已从 {input_path} 加载")
        return data


print("✓ StockDataFetcher类定义完成")

## 📊 Step 6: 执行数据抓取

In [ ]:
# 创建数据抓取器
fetcher = StockDataFetcher()

# ============================================================================
# 配置数据抓取参数
# ============================================================================

START_DATE = "2020-01-01"  # 开始日期
END_DATE = "2024-12-31"    # 结束日期
INTERVAL = "1d"            # "1d" 按天，"1h" 按小时

# 分批和延迟配置（使用默认推荐值）
BATCH_SIZE = 5              # 每批5支股票
DELAY_BETWEEN_BATCHES = 2.0  # 批次间延迟2秒
DELAY_BETWEEN_STOCKS = 0.5   # 股票间延迟0.5秒

# ============================================================================
# 抓取数据
# ============================================================================

historical_data = fetcher.fetch_historical_data(
    stocks_json=agent.selected_stocks,
    start_date=START_DATE,
    end_date=END_DATE,
    interval=INTERVAL,
    include_market_index=True,
    batch_size=BATCH_SIZE,
    delay_between_batches=DELAY_BETWEEN_BATCHES,
    delay_between_stocks=DELAY_BETWEEN_STOCKS
)

# 保存数据
fetcher.save_data("historical_data.pkl")

print("\n" + "="*80)
print("\n数据统计:")
for market, stocks_data in historical_data.items():
    print(f"\n{market}市场:")
    for symbol, df in stocks_data.items():
        print(f"  {symbol}: {len(df)}条记录")

---

## ✅ 智能体部分完成

现在你已经拥有：
1. ✓ 股票选择JSON文件 (`selected_stocks.json`)
2. ✓ 分析报告 (`analysis_report.md`)
3. ✓ 历史数据 (`historical_data.pkl`)

**下一步**：
- 继续运行后续cells进行模型训练和验证
- 或在新的notebook中加载这些数据进行分析

---